# 📊 Análise de Pacientes Adultos em Diálise - Porto Alegre (RS)
Este notebook analisa dados de pacientes adultos (18+) em diálise com foco em internações, perfil demográfico e variações ao longo do tempo.

In [29]:
# 📦 Importação de bibliotecas
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

In [30]:
# 📂 Leitura dos dados
df = pd.read_csv('dados_brutos.csv', encoding='latin1')
municipios_rs = pd.read_csv('municipios_rs.csv', dtype={'codigo_ibge': str})

C:\Users\arthu\AppData\Local\Temp\ipykernel_20052\2657273389.py:2: DtypeWarning:

Columns (20,37,44) have mixed types. Specify dtype option on import or set low_memory=False.



In [31]:
# 🧹 Tratamento inicial
# Código do município
df['AP_MUNPCN'] = df['AP_MUNPCN'].astype(str).str.lstrip('0')
municipios_rs['codigo_mun6'] = municipios_rs['codigo_ibge'].str[:-1]  # descarta dígito verificador
df = df.merge(municipios_rs[['codigo_mun6', 'nome']], left_on='AP_MUNPCN', right_on='codigo_mun6', how='left')
df = df.rename(columns={'nome': 'MUNICIPIO'})

In [32]:
print(df.columns.tolist())

['AP_MVM', 'AP_CONDIC', 'AP_GESTAO', 'AP_CODUNI', 'AP_AUTORIZ', 'AP_CMP', 'AP_PRIPAL', 'AP_VL_AP', 'AP_UFMUN', 'AP_TPUPS', 'AP_TIPPRE', 'AP_MN_IND', 'AP_CNPJCPF', 'AP_CNPJMNT', 'AP_CNSPCN', 'AP_COIDADE', 'AP_NUIDADE', 'AP_SEXO', 'AP_RACACOR', 'AP_MUNPCN', 'AP_UFNACIO', 'AP_CEPPCN', 'AP_UFDIF', 'AP_MNDIF', 'AP_DTINIC', 'AP_DTFIM', 'AP_TPATEN', 'AP_TPAPAC', 'AP_MOTSAI', 'AP_OBITO', 'AP_ENCERR', 'AP_PERMAN', 'AP_ALTA', 'AP_TRANSF', 'AP_DTOCOR', 'AP_CODEMI', 'AP_CATEND', 'AP_APACANT', 'AP_UNISOL', 'AP_DTSOLIC', 'AP_DTAUT', 'AP_CIDCAS', 'AP_CIDPRI', 'AP_CIDSEC', 'AP_ETNIA', 'ATD_CARACT', 'ATD_DTPDR', 'ATD_DTCLI', 'ATD_ACEVAS', 'ATD_MAISNE', 'ATD_SITINI', 'ATD_SITTRA', 'ATD_SEAPTO', 'ATD_HB', 'ATD_FOSFOR', 'ATD_KTVSEM', 'ATD_TRU', 'ATD_ALBUMI', 'ATD_PTH', 'ATD_HIV', 'ATD_HCV', 'ATD_HBSAG', 'ATD_INTERC', 'ATD_SEPERI', 'arquivo_origem', 'AP_NATJUR', 'codigo_mun6', 'MUNICIPIO']


In [33]:
# Exemplo 1: Contagem por município (ajuste o nome da coluna)
top_municipios = df['MUNICIPIO'].value_counts().head(10).reset_index()
top_municipios.columns = ['Município', 'Quantidade']

fig = px.bar(
    top_municipios,
    x='Quantidade',
    y='Município',
    orientation='h',
    title='Top 10 Municípios com mais Registros de Diálise',
    text='Quantidade'
)
fig.update_layout(yaxis={'categoryorder': 'total ascending'})
fig.show()


In [34]:
# Filtro para pacientes adultos no Estado (idade em anos)
df_adultos = df[(df['AP_COIDADE'] == 4) & (df['AP_NUIDADE'] >= 18)]
print(f'Total de pacientes adultos: {len(df_adultos)}')

Total de pacientes adultos: 785634


In [35]:
# 📆 Extração do ano da competência
df['ano'] = df['AP_CMP'].astype(str).str[:4]

In [36]:
# 🧑‍⚕️ Filtra apenas pacientes adultos (idade em anos)
df_adultos = df[(df['AP_COIDADE'] == 4) & (df['AP_NUIDADE'] >= 18)]
print(f'Total de pacientes adultos: {len(df_adultos)}')

Total de pacientes adultos: 785634


In [37]:
# 📍 Filtra pacientes com internação em Porto Alegre
df_poa = df_adultos[df_adultos['AP_MUNPCN'] == '431490'].copy()
print(f'Total de pacientes adultos: {len(df_poa)}')

Total de pacientes adultos: 115851


In [38]:
# 👵 Criação de faixas etárias
bins = [18, 30, 45, 60, 75, 90, 200]
labels = ['18-29', '30-44', '45-59', '60-74', '75-89', '90+']
df_poa['faixa_etaria'] = pd.cut(df_poa['AP_NUIDADE'], bins=bins, labels=labels, right=False)

In [39]:
# 🧠 Mapeia sexo para nomes completos
df_poa['sexo_label'] = df_poa['AP_SEXO'].map({'M': 'Masculino', 'F': 'Feminino'})

In [40]:
# 📊 Distribuição por sexo
sexo_counts = df_poa['sexo_label'].value_counts().reset_index()
sexo_counts.columns = ['Sexo', 'Quantidade']

fig = px.bar(
    sexo_counts,
    x='Sexo',
    y='Quantidade',
    color='Sexo',
    text='Quantidade',
    title='Distribuição por Sexo - Pacientes em Diálise POA'
)
fig.show()

In [41]:
# 📊 Distribuição por faixa etária
faixa_counts = df_poa['faixa_etaria'].value_counts().sort_index().reset_index()
faixa_counts.columns = ['Faixa Etária', 'Quantidade']

fig = px.bar(
    faixa_counts,
    x='Faixa Etária',
    y='Quantidade',
    text='Quantidade',
    title='Distribuição por Faixa Etária - Pacientes em Diálise POA'
)
fig.show()


In [42]:
# 🎨 Visualização: distribuição por sexo e faixa etária

fig = px.histogram(
    df_poa,
    x='faixa_etaria',
    color='sexo_label',
    barmode='group',
    title='Distribuição por Faixa Etária e Sexo - Pacientes em Diálise POA',
    labels={'faixa_etaria': 'Faixa Etária', 'sexo_label': 'Sexo'}
)
fig.update_layout(xaxis_title='Faixa Etária', yaxis_title='Número de Pacientes')
fig.show()


In [43]:
# 📅 Distribuição por ano
ano_counts = df_poa['ano'].value_counts().sort_index().reset_index()
ano_counts.columns = ['Ano', 'Quantidade']

fig = px.bar(
    ano_counts,
    x='Ano',
    y='Quantidade',
    text='Quantidade',
    title='Distribuição de Pacientes em Diálise por Ano (POA)'
)
fig.show()


In [44]:
# 🏥 Locais de realização da diálise
top_locais = df_poa['AP_CODUNI'].value_counts().head(10).reset_index()
top_locais.columns = ['Código da Unidade', 'Quantidade']

fig = px.bar(
    top_locais,
    x='Quantidade',
    y='Código da Unidade',
    orientation='h',
    title='Top 10 Locais de Realização de Diálise - POA',
    text='Quantidade'
)
fig.update_layout(yaxis={'categoryorder': 'total ascending'})
fig.show()


In [45]:
# 🧮 Tabela cruzada por faixa etária e ano
# Tabela cruzada
tabela_heat = pd.crosstab(df_poa["faixa_etaria"], df_poa["ano"])

# Geração do heatmap interativo
fig = px.imshow(
    tabela_heat,
    text_auto=True,
    labels=dict(x="Ano", y="Faixa Etária", color="Quantidade"),
    title="Internações por Faixa Etária e Ano - Pacientes em Diálise (POA)"
)
fig.update_layout(
    xaxis_side="top",
    yaxis=dict(autorange="reversed")
)
fig.show()

In [46]:
# 👥 Comparativo de idade média por unidade
# Cálculo da média de idade por unidade
media_idade_unidade = df_poa.groupby("AP_CODUNI")["AP_NUIDADE"].mean().sort_values(ascending=False).head(10)
media_df = media_idade_unidade.reset_index()
media_df.columns = ["Unidade", "Idade Média"]

# Geração do gráfico de barras interativo
fig = px.bar(
    media_df,
    x="Idade Média",
    y="Unidade",
    orientation="h",
    title="Idade Média dos Pacientes por Unidade de Diálise - POA"
)

fig.update_layout(yaxis=dict(autorange="reversed"))
fig.show()

In [47]:
# 🏥 Tabela resumo por unidade de atendimento
resumo_unidades = df_poa.groupby('AP_CODUNI').agg(
    pacientes=('AP_CNSPCN', 'nunique'),
    idade_media=('AP_NUIDADE', 'mean'),
    percentual_feminino=('AP_SEXO', lambda x: (x == 'F').mean() * 100)
).sort_values(by='pacientes', ascending=False)
resumo_unidades.head(10)

,pacientes,idade_media,percentual_feminino
AP_CODUNI,,,
2262509,3009,58.749787,39.440756
2237253,1432,54.823569,47.574282
2237601,1254,57.334399,50.729562
2262460,894,54.207930,48.079434
2262584,816,60.383475,44.610557
2262770,671,58.915114,41.264651
2262568,518,56.200094,44.694584
2237571,344,59.606027,46.428571
2237598,154,55.539213,47.114531


In [48]:
cidcas_map = {
    "0000": "Não especificado",
    "B182": "Hepatite C crônica",
    "N180": "Doença renal em estágio final",
    "Z21": "Portador assintomático do HIV",
    "B171": "Hepatite C aguda",
    "B181": "Hepatite B crônica",
    "B200": "Doença pelo HIV resultando em infecção micobacteriana",
    "B202": "Doença pelo HIV resultando em encefalopatia",
    "B24": "Doença pelo HIV não especificada",
    "B160": "Hepatite B aguda com delta (coinfecção)",
    "B180": "Hepatite B crônica",
    "N189": "Doença renal crônica, estágio não especificado",
    "B189": "Hepatite viral crônica, não especificada",
    "B170": "Hepatite B aguda",
    "B162": "Hepatite B aguda sem delta",
    "B188": "Outras hepatites virais crônicas",
    "B217": "Doença pelo HIV com outras doenças virais",
    "B012": "Varicela com encefalite",
    "B178": "Outras hepatites virais agudas",
    "B207": "Doença pelo HIV com cânceres múltiplos",
    "E142": "Diabetes mellitus secundário com complicações renais",
    "N188": "Outras doenças renais crônicas especificadas",
    "I120": "Doença cardíaca hipertensiva com insuficiência renal",
    "I10": "Hipertensão essencial (primária)",
    "C79": "Metástase de tumor maligno em outros órgãos",
    "B18": "Hepatite viral crônica",
    "B16": "Hepatite B aguda"
}

df_poa["CIDCAS_DESC"] = df_poa["AP_CIDCAS"].map(cidcas_map).fillna("Outro/Não mapeado")


In [49]:
# 🧾 Top 10 CIDs de causa associada mais frequentes
top_cidcas = df_poa["CIDCAS_DESC"].value_counts().head(10)
cidcas_df = top_cidcas.reset_index()
cidcas_df.columns = ["CID", "Ocorrências"]

# Gráfico interativo
fig = px.bar(
    cidcas_df,
    x="Ocorrências",
    y="CID",
    orientation="h",
    title="Top 10 CIDs de Causa Associada em Pacientes em Diálise (POA)"
)

fig.update_layout(yaxis=dict(autorange="reversed"))
fig.show()





In [50]:
# 🧾 Top 10 CIDs de causa associada mais frequentes (sem códigos inválidos)
# Filtra e conta as CIDs, excluindo "0000"
top_cidcas = df_poa[df_poa["AP_CIDCAS"] != "0000"]["CIDCAS_DESC"].value_counts().head(10)
cidcas_df = top_cidcas.reset_index()
cidcas_df.columns = ["CID", "Ocorrências"]

# Gráfico com Plotly
fig = px.bar(
    cidcas_df,
    x="Ocorrências",
    y="CID",
    orientation="h",
    title="Top 10 CIDs de Causa Associada em Pacientes em Diálise (POA)",
)

fig.update_layout(yaxis=dict(autorange="reversed"))
fig.show()




In [51]:
mapa_procedimentos = {
    '305010107.0': 'Hemodiálise (3x/sem)',
    '305010115.0': 'Hemodiálise HIV/Hepatite',
    '305010166.0': 'Manutenção DPA/DPAC',
    '305010182.0': 'Treinamento DPAC-DPA',
    '305010026.0': 'Diálise Peritoneal Intermitente'
}

df_poa['procedimento_desc'] = df_poa['AP_PRIPAL'].astype(str).map(mapa_procedimentos)

In [52]:
# Filtra e conta os procedimentos válidos
top_procs = (
    df_poa[df_poa["procedimento_desc"].notna()]["procedimento_desc"]
    .value_counts()
    .reset_index()
)
top_procs.columns = ["Procedimento", "Ocorrências"]

# Gráfico interativo
fig = px.bar(
    top_procs,
    x="Ocorrências",
    y="Procedimento",
    orientation="h",
    title="Top Procedimentos em Pacientes em Diálise (POA)",
)

fig.update_layout(yaxis=dict(autorange="reversed"))
fig.show()

In [53]:
hospital_mapping_dict ={
    # Código: "Nome do Estabelecimento"
    2262509: "SER SERVICO DE DOENCAS RENAIS LTDA",
    2237253: "IRMANDADE DA SANTA CASA DE MISERICORDIA DE PORTO ALEGRE",
    2262584: "CLINIRIM",
    2262770: "CENTRO DE DIALISE E TRANSPLANTE LTDA", # Também listado como TRANSPLANTAR LTDA
    2262460: "VITARIM SERVICO DO RIM",
    2237601: "HOSPITAL DE CLINICAS", # Porto Alegre
    2262568: "HOSPITAL SAO LUCAS DA PUCRS",
}

cnes_to_address = {
    2262509: "R CATARINO ANDREATTA, 155",
    2237253: "R PROF ANNES DIAS, 295",
    2262584: "RUA SPORT CLUB SAO JOSE, 222",
    2262770: "AV VISCONDE DO RIO BRANCO, 755",
    2262460: "RUA DA CONCEICAO, 560",
    2237601: "RUA RAMIRO BARCELOS, 2350",
    2262568: "AV IPIRANGA, 6690"
}

df_poa['NOME_HOSPITAL'] = df_poa['AP_CODUNI'].map(hospital_mapping_dict)
df_poa['ENDERECO'] = df_poa['AP_CODUNI'].map(cnes_to_address)

In [54]:
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

# 1. Crie um DataFrame apenas com os endereços únicos para evitar requisições repetidas
df_com_enderecos = df_poa.dropna(subset=['ENDERECO'])
df_enderecos_unicos = df_com_enderecos[['ENDERECO']].drop_duplicates().reset_index(drop=True)

# 2. Inicialize o geocodificador com um RateLimiter para respeitar a política de uso
geolocator = Nominatim(user_agent="analise_hospitalar_poa_v2")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

# 3. Crie a função de geocodificação
def get_coords(row):
    # Adicionamos o estado e país para ajudar o geocodificador a encontrar o local correto
    address_to_find = f"{row['ENDERECO']}, PORTO ALEGRE, RS, Brazil"
    try:
        location = geocode(address_to_find, timeout=10)
        if location:
            return pd.Series((location.latitude, location.longitude))
        else:
            return pd.Series((None, None))
    except Exception as e:
        print(f"Erro geocodificando {address_to_find}: {e}")
        return pd.Series((None, None))

# 4. Aplique a função ao DataFrame de endereços únicos
print("\nIniciando geocodificação dos endereços únicos...")
df_enderecos_unicos[['latitude', 'longitude']] = df_enderecos_unicos.apply(get_coords, axis=1)
print("Geocodificação concluída.")
print("\nEndereços únicos com coordenadas:")
print(df_enderecos_unicos)


# 5. Junte (merge) as coordenadas de volta ao DataFrame principal
df_final = pd.merge(df_poa, df_enderecos_unicos, on='ENDERECO', how='left')


Iniciando geocodificação dos endereços únicos...
Geocodificação concluída.

Endereços únicos com coordenadas:
                         ENDERECO   latitude  longitude
0          R PROF ANNES DIAS, 295 -30.030895 -51.223580
1               AV IPIRANGA, 6690 -30.055026 -51.175780
2       RUA RAMIRO BARCELOS, 2350 -30.038678 -51.206625
3           RUA DA CONCEICAO, 560 -30.032510 -51.219347
4       R CATARINO ANDREATTA, 155 -30.119452 -51.207765
5  AV VISCONDE DO RIO BRANCO, 755 -30.016098 -51.203960
6    RUA SPORT CLUB SAO JOSE, 222 -30.010072 -51.170779


In [55]:
df_final.to_csv('dados_tratados.csv')

In [56]:
ap_cmp_counts = df_poa['AP_CODUNI'].value_counts()

# Print the full list of counts
print(ap_cmp_counts)

AP_CODUNI
2262509    23496
2237253    17871
2262584    16254
2262770    16210
2262460    14553
2237601    10006
2262568     6352
2237598     3379
5844762     2791
2231042     1483
2237571      896
2237164      883
2231069      524
2264137      333
2257815      192
2707829      155
2231034      135
2232022       56
2231778       54
2242397       39
2253046       31
2232049       19
2263858       13
2247984       12
2223546       11
2254611       11
2232995       11
2227843       10
2256029        7
2261057        7
2247429        7
2246929        7
2246988        6
2247771        6
2248204        4
2248298        4
2232472        3
2244357        2
2241048        2
2247968        2
7288239        2
2252295        2
2244306        2
2261898        2
2236370        1
2253054        1
2233312        1
2248220        1
2229706        1
2248328        1
Name: count, dtype: int64
